# Transform windows only vrt to cross-platform


## Imports

In [ ]:
import pandas as pd
from pathlib import Path
import itertools
import shutil

from utils_report import *

## Setup 
* paths

In [ ]:
# setup basepaths
DIR_BASE = Path(r'S:\p_macsprocessing')
DIR_DATA_PRODUCTS = DIR_BASE / 'data_products'

In [ ]:
# check if directories all exists
for d in [DIR_BASE, DIR_DATA_PRODUCTS]:
    assert d.exists()

## Calculate Statistics 
* Files
* File Count
* file count accross types
* aoi (size?)


#### Setup basic Dataframe and split input name

In [ ]:
df = pd.DataFrame(columns=['project_name', 'products_dir'])
# create pathlist of output products
dir_list = list(DIR_DATA_PRODUCTS.glob('*'))
df['products_dir'] = dir_list
# get project name
df['project_name'] = df['products_dir'].apply(lambda x: x.name)
# add site specific details
df = split_name_details(df)

In [ ]:
#check ortho, dsm and processing_info
file_check_columns = ['DSM', 'Ortho','processing_info']
cols_file_check = flatten([[f"{item}_dir_exists", f"{item}_n_files"] for item in file_check_columns])

file_check_output = df.apply(file_check, dirs=file_check_columns, axis=1)
df = df.join(pd.DataFrame(file_check_output.to_list(), columns=cols_file_check))

In [ ]:
#check for base files
# has vrt files
df['vrt_exists'] = df.apply(check_files_vrt, axis=1)

#### Read vrt

In [ ]:
df['vrt_is_linux'] = df.apply(check_vrt_is_linux, axis=1)

In [ ]:
def transform_files_vrt(row):
    for item in ['Ortho.vrt', 'DSM.vrt']:
        vrt_file = row['products_dir'] / item
        vrt_transform_win_to_linux(vrt_file, backup=False)

### Run file transformation 

In [ ]:
df.iloc[:].apply(transform_files_vrt, axis=1)